# Main class for running the molecule environment

In [1]:
import gym
from gym import wrappers, logger
from MoleculeAgent import MoleculeAgent

In [2]:
def main():
    env = gym.make("gym_molecule:molecule-v0")
    
    #Create a new Molecule Agent object. 
    agent = MoleculeAgent(env.action_space)
    
    episode_count = 3
    reward = 0
    done = False

    #The 'RL algorithm' learns for 2 episodes of undefined length.
    for i in range(0, episode_count):
        print("EPISODE:", i)
        
        #The env resets.
        ob = env.reset()
        print("Env original state: ", ob)
        
        #While what is True? Maybe While done is False?
        for x in range(20):
            
            #The agent chooses its action based on the tuple from the current env state.
            action = agent.act(ob, reward, done)
            print()
            print("Agent's chosen action: ", action) 
            
            #The new tuple is returned because the env has considered the agent's action.
            ob, reward, done, _ = env.step(action)
            print("Env new state:", ob, "Reward to agent:",reward, "Done:",done, "Info: ", _)
            
            #The env renders. 
            env.render()
            
            
        
            #If the env reaches a terminating state, the current iteration ends and the next 
            #episode begins. 
            """"if done:
                print()
                break"""
        
    env.close()
    
    
if __name__ == '__main__':
    main()

Enter The Optimisation goal: 
 eg 'FC(F)(Cl)C(F)(Cl)Cl'1
EPISODE: 0
Env original state:  None

Agent's chosen action:  4
add bracketed atom
['bracketed atom']
calculating Reward
Env new state: 0.0 Reward to agent: 1 Done: False Info:  {}

Agent's chosen action:  0
['bracketed atom', 'S']
calculating Reward
Env new state: 0.0 Reward to agent: 1 Done: False Info:  {}

Agent's chosen action:  0
['bracketed atom', 'S', 'O']
calculating Reward
Env new state: 0.0 Reward to agent: 1 Done: False Info:  {}

Agent's chosen action:  1
['bracketed atom', 'S', 'O', 'P']
calculating Reward
Env new state: 1.2987 Reward to agent: 1 Done: False Info:  {}

Agent's chosen action:  3
['bracketed atom', 'S', 'O', 'P', 'O']
calculating Reward
Env new state: 3.7975 Reward to agent: 1 Done: False Info:  {}

Agent's chosen action:  2
['bracketed atom', 'S', 'O', 'P', 'O', '=S']
calculating Reward
Env new state: 3.7975 Reward to agent: 1 Done: False Info:  {}

Agent's chosen action:  3
['bracketed atom', 'S', '

RDKit ERROR: [21:04:23] Explicit valence for atom # 4 O, 3, is greater than permitted
RDKit ERROR: [21:04:23] Explicit valence for atom # 1 O, 3, is greater than permitted
RDKit ERROR: [21:04:23] Explicit valence for atom # 5 O, 3, is greater than permitted
RDKit ERROR: [21:04:23] Explicit valence for atom # 1 O, 4, is greater than permitted
RDKit ERROR: [21:04:23] Explicit valence for atom # 1 O, 3, is greater than permitted
RDKit ERROR: [21:04:23] Explicit valence for atom # 1 O, 4, is greater than permitted
RDKit ERROR: [21:04:23] Explicit valence for atom # 1 O, 3, is greater than permitted
RDKit ERROR: [21:04:23] Explicit valence for atom # 1 O, 3, is greater than permitted
RDKit ERROR: [21:04:23] Explicit valence for atom # 6 O, 3, is greater than permitted
RDKit ERROR: [21:04:23] Explicit valence for atom # 6 O, 4, is greater than permitted
RDKit ERROR: [21:04:23] Explicit valence for atom # 1 O, 3, is greater than permitted
